In [4]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost3	 = Beta('Cost3',0,None,None,0)
Cost4	 = Beta('Cost4',0,None,None,0)
Total_TT1	 = Beta('Total_TT1',0,None,None,0)
Total_TT2	 = Beta('Total_TT2',0,None,None,0)
Total_TT3	 = Beta('Total_TT3',0,None,None,0)
Total_TT4	 = Beta('Total_TT4',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT1 * dur_walking
Opt2 = Constant2 + Total_TT2 * dur_cycling
Opt3 = Constant3 + Cost3 * cost_public + Total_TT3 * dur_public
Opt4 = Constant4 + Cost4 * cost_driving + Total_TT4 * dur_driving
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model1_cost_time"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

           Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2  -4.61    0.172   -26.8        0         0.214        -21.5   
Constant3  -2.27    0.115   -19.6        0          0.15        -15.1   
Constant4  -1.88    0.107   -17.5        0          0.15        -12.5   
Cost3     -0.163    0.029   -5.64 1.73e-08        0.0302        -5.41   
Cost4     -0.153   0.0169   -9.04        0         0.016        -9.53   
Total_TT1  -8.19    0.294   -27.9        0         0.455          -18   
Total_TT2  -5.87    0.452     -13        0         0.518        -11.3   
Total_TT3   -3.6    0.236   -15.3        0         0.242        -14.9   
Total_TT4  -6.54    0.368   -17.8        0         0.412        -15.9   

           Rob. p-value  
Constant2             0  
Constant3             0  
Constant4             0  
Cost3          6.24e-08  
Cost4                 0  
Total_TT1             0  
Total_TT2             0  
Total_TT3             0  
Total_TT4             0  
Nbr of ob